# TOC
1. [Settings](#Settings)
2. [Get the task list](#Get-the-task-list)
3. [Upload annotations](#Upload-annotations)
4. [Get annotation results](#Get-annotation-results)
5. [Get annotation detail log](#Get-annotation-detail-log)

# Settings

In [27]:
import init
import pandas as pd
import json
import requests

In [28]:
host = 'http://api:5000'
headers = {}

In [14]:
# Cloud Run API needs Authorization
host = 'https://******.a.run.app'
headers = {
    'Authorization': 'Bearer <TOKEN>'
}

# Get the task list

In [29]:
res = requests.get(f'{host}/tasks', headers=headers).json()
pd.DataFrame(res)[:3]

,description,annotation_type,title,id,created_at,question,updated_at
0,"This is a card demo, so feel free to annotate it as you wish.",card,Card Demo,card-demo-20200602,2021-06-01T13:52:45.882546+00:00,This is card demo,2021-06-01T13:52:45.882564+00:00
1,"This is a card demo, so feel free to annotate it as you wish.",card,Card Demo,card-demo-20200603,2021-06-02T04:48:38.511052+00:00,This is card demo,2021-06-02T04:48:38.511062+00:00


# Upload annotations

## Card UI

- [Task-dependent schema#card](https://github.com/CyberAgent/fast-annotation-tool/wiki/%E3%82%BF%E3%82%B9%E3%82%AF%E4%BE%9D%E5%AD%98%E3%81%AE%E3%82%B9%E3%82%AD%E3%83%BC%E3%83%9E#card)

<img src="https://user-images.githubusercontent.com/17490886/121502506-33633700-ca1b-11eb-9bf2-012b58aee50a.png" alt="image" />

In [38]:
# Make annotation data
annotations_data = [
    {
        "text": f"This is a test{i}.", 
        "show_ambiguous_button": True, 
        "hidden_data": {
            "desc": "Data for aggregation. It can be a dictionary or a string."
        }
     } for i in range(100)
]
df_annotation = pd.DataFrame(annotations_data)
df_annotation[:3]

,text,show_ambiguous_button,hidden_data
0,This is a test0.,True,{'desc': 'Data for aggregation. It can be a dictionary or a string.'}
1,This is a test1.,True,{'desc': 'Data for aggregation. It can be a dictionary or a string.'}
2,This is a test2.,True,{'desc': 'Data for aggregation. It can be a dictionary or a string.'}


In [39]:
# Post task data
post_data = {
    "task_id": "card-demo-20200602",
    "annotation_type": "card",
    "title": "Card Demo",
    "question": "This is card demo",
    "description": "This is a card demo, so feel free to annotate it as you wish.",
    "annotations_data": annotations_data
}
res = requests.post(f'{host}/tasks', headers=headers, json=post_data).json()
res

{'errors': [{'loc': ['body', 'task_id'],
   'msg': 'task_id: card-demo-baseline-20200602 は既に存在します.',
   'type': 'value_error'}]}

## Multi-Label UI

- [Task-dependent schema#multilabel](https://github.com/CyberAgent/fast-annotation-tool/wiki/%E3%82%BF%E3%82%B9%E3%82%AF%E4%BE%9D%E5%AD%98%E3%81%AE%E3%82%B9%E3%82%AD%E3%83%BC%E3%83%9E#multilabel)


<img src="https://user-images.githubusercontent.com/17490886/121502256-f4cd7c80-ca1a-11eb-9429-5ee52c55d949.png" alt="image" />

In [40]:
# Make annotation data
annotation_data = [
    {
        "text": f"This is a test{i}.",
        "choices": ["ChoiceA", "ChoiceB", "ChoiceC", "ChoiceD"],
        "baseline_text": "Baseline Text",
        "hidden_data": {
            "desc": "Data for aggregation. It can be a dictionary or a string."
        }
    }
    for i in range(100)
]
df_annotation = pd.DataFrame(annotation_data)
df_annotation[:3]

,text,choices,baseline_text,hidden_data
0,This is a test0.,"[ChoiceA, ChoiceB, ChoiceC, ChoiceD]",Baseline Text,{'desc': 'Data for aggregation. It can be a dictionary or a string.'}
1,This is a test1.,"[ChoiceA, ChoiceB, ChoiceC, ChoiceD]",Baseline Text,{'desc': 'Data for aggregation. It can be a dictionary or a string.'}
2,This is a test2.,"[ChoiceA, ChoiceB, ChoiceC, ChoiceD]",Baseline Text,{'desc': 'Data for aggregation. It can be a dictionary or a string.'}


In [36]:
# Post task data
post_data = {
    "task_id": "multilabel-demo-20200602",
    "annotation_type": "multi_label",
    "title": "Multi-Label Demo",
    "question": "This is multi-label demo",
    "description": "This is a multi-label demo, so feel free to annotate it as you wish.",
    "annotations_data": annotation_data
}
res = requests.post(f'{host}/tasks', headers=headers, json=post_data).json()
res

{'message': 'Success upload.',
 'task_id': 'multilabel-demo-20200602',
 'annotation_num': 100,
 'task_url': 'https://cyberagent-342.web.app/task/multilabel-demo-20200602'}

# Get annotation results

In [65]:
%%time
task_id = "card-demo-20200602"
res = requests.get(f'{host}/tasks/{task_id}', headers=headers).json()

CPU times: user 4.73 ms, sys: 2.57 ms, total: 7.3 ms
Wall time: 1.65 s


In [60]:
# Task Info
res['task']

{'id': 'card-demo-20200602',
 'annotation_type': 'card',
 'title': 'Card Demo',
 'question': 'This is card demo',
 'description': 'This is a card demo, so feel free to annotate it as you wish.',
 'created_at': '2021-06-01T13:52:45.882546+00:00',
 'updated_at': '2021-06-01T13:52:45.882564+00:00'}

In [62]:
# Annotation data and annotator responses
df_res =  pd.DataFrame(res['annotations'])
df_res['name'] = '****'
df_res['email'] = '****'
df_res[~df_res.result_data.isna()][:3]

,id,name,email,data,result_data,order_index,user_id,user_task_id,annotation_id,created_at,updated_at
2,0XrLJijwR8YEAlKOzawu,****,****,"{'baseline_text': None, 'show_ambiguous_button': True, 'question_overwrite': None, 'text': 'This is a test1.', 'hidden_data': {'desc': 'Data for aggregation. It can be a dictionary or a string.'}, 'yes_button_label': None, 'no_button_label': None}",{'result': 'Yes'},2,SqZ1hzgybtUI9jK7aSCAJ1g8HlA2,FAiEcGphh1NMawdOHbMn,Szk8f6VNalIctqTZXDrC,2021-06-02T12:18:21.689000+00:00,2021-06-02T12:19:27.263000+00:00
9,4aeAH2uNO2V4MO7hVyBU,****,****,"{'hidden_data': {'desc': 'Data for aggregation. It can be a dictionary or a string.'}, 'text': 'This is a test7.', 'question_overwrite': None, 'show_ambiguous_button': True, 'baseline_text': None, 'no_button_label': None, 'yes_button_label': None}",{'result': 'Yes'},8,SqZ1hzgybtUI9jK7aSCAJ1g8HlA2,FAiEcGphh1NMawdOHbMn,YRYGUV6SiG1KX2tE1OZo,2021-06-02T12:18:21.689000+00:00,2021-06-02T12:19:33.523000+00:00
12,5n4uKKb7rpVNNOUsRpjK,****,****,"{'no_button_label': None, 'hidden_data': {'desc': 'Data for aggregation. It can be a dictionary or a string.'}, 'question_overwrite': None, 'show_ambiguous_button': True, 'baseline_text': None, 'text': 'This is a test3.', 'yes_button_label': None}",{'result': 'Yes'},4,SqZ1hzgybtUI9jK7aSCAJ1g8HlA2,FAiEcGphh1NMawdOHbMn,i4g0otOehmF6EBasZtxJ,2021-06-02T12:18:21.689000+00:00,2021-06-02T12:19:30.857000+00:00


# Get annotation detail log

In [66]:
task_id = "card-demo-20200602"
res = requests.get(f'{host}/tasks/{task_id}/logs', headers=headers).json()

In [73]:
df_res =  pd.DataFrame(res['logs'])
df_res['name'] = '****'
df_res['email'] = '****'
df_res.sample(5)

,id,name,email,data,action_type,action_data,user_task_id,user_annotation_id,created_at
6,FPAgQwLpo7jSK57XmVcY,****,****,None,display,"{'deviceInfo': {'view': {'pixelDepth': 30, 'innerWidth': 786, 'availHeight': 875, 'innerHeight': 884, 'colorDepth': 30, 'screenHeight': 900, 'screenWidth': 1440, 'availWidth': 1440}, 'navigator': {'userAgent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 'maxTouchPoints': '0', 'appName': 'Netscape', 'appCodeName': 'Mozilla', 'pointerEnabled': 'undefined', 'platform': 'MacIntel', 'vendor': 'Google Inc.', 'language...",FAiEcGphh1NMawdOHbMn,FeIY6C3yTmsiGTHNqhPw,2021-06-02T12:19:27.938000+00:00
2,5N1TPclWlmd4LjRMrNuj,****,****,None,display,"{'deviceInfo': {'navigator': {'product': 'Gecko', 'vendorSub': '', 'platform': 'MacIntel', 'appCodeName': 'Mozilla', 'pointerEnabled': 'undefined', 'productSub': '20030107', 'maxTouchPoints': '0', 'appName': 'Netscape', 'languages': ['ja', 'en-US', 'en'], 'userAgent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 'vendor': 'Google Inc.', 'appVersion': '5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KH...",FAiEcGphh1NMawdOHbMn,5n4uKKb7rpVNNOUsRpjK,2021-06-02T12:19:30.759000+00:00
16,ptLPPOM5lrlS21zoqOFw,****,****,None,display,"{'deviceInfo': {'view': {'pixelDepth': 30, 'availHeight': 875, 'screenWidth': 1440, 'innerWidth': 786, 'innerHeight': 884, 'screenHeight': 900, 'availWidth': 1440, 'colorDepth': 30}, 'location': {'href': 'http://localhost:8080/user_task/FAiEcGphh1NMawdOHbMn/annotation/4aeAH2uNO2V4MO7hVyBU', 'referrer': '', 'domain': 'localhost'}, 'navigator': {'platform': 'MacIntel', 'productSub': '20030107', 'product': 'Gecko', 'vendor': 'Google Inc.', 'appCodeName': 'Mozilla', 'maxTouchPoints': '0', 'userA...",FAiEcGphh1NMawdOHbMn,p4WsKw8O3p2xXBGys3gE,2021-06-02T12:19:32.821000+00:00
18,vERP165YnmKcIAlEWMps,****,****,None,display,"{'deviceInfo': {'view': {'screenWidth': 1440, 'innerHeight': 884, 'screenHeight': 900, 'availWidth': 1440, 'innerWidth': 786, 'colorDepth': 30, 'availHeight': 875, 'pixelDepth': 30}, 'location': {'href': 'http://localhost:8080/user_task/FAiEcGphh1NMawdOHbMn/annotation/p4WsKw8O3p2xXBGys3gE', 'referrer': '', 'domain': 'localhost'}, 'navigator': {'product': 'Gecko', 'languages': ['ja', 'en-US', 'en'], 'platform': 'MacIntel', 'vendor': 'Google Inc.', 'pointerEnabled': 'undefined', 'vendorSub': '...",FAiEcGphh1NMawdOHbMn,EhZ9XIJEC0LMqu5PG5R5,2021-06-02T12:19:32.083000+00:00
7,H2mIYjHvY7Vp3lPHGDDN,****,****,None,submit,"{'deviceInfo': {'view': {'innerWidth': 786, 'availWidth': 1440, 'availHeight': 875, 'innerHeight': 884, 'pixelDepth': 30, 'screenHeight': 900, 'colorDepth': 30, 'screenWidth': 1440}, 'location': {'domain': 'localhost', 'referrer': '', 'href': 'http://localhost:8080/user_task/FAiEcGphh1NMawdOHbMn/annotation/XWegDattSoj4Kvv2AeI6'}, 'navigator': {'pointerEnabled': 'undefined', 'userAgent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.2...",FAiEcGphh1NMawdOHbMn,XWegDattSoj4Kvv2AeI6,2021-06-02T12:20:47.923000+00:00
